In [116]:
# import the used libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import QuantLib as ql
import BootStrap.BootStrapping as bs
import MarketData.MarketData as md
import  math
#import MarketData.ReadExcel 
#import openpyxl 
import re

%matplotlib inline



In [117]:
# Setting evaluation date
today = ql.Date.todaysDate()
#today = ql.TARGET().adjust(today,ql.Following)

ql.Settings.instance().setEvaluationDate(today)


In [118]:
mc = md.MarketConventions()
mdata = md.getBBMarketData()


oisZeroYields = bs.BootstrapOIS(mdata,mc)
discountTermStructure = ql.RelinkableYieldTermStructureHandle()
discountTermStructure.linkTo(oisZeroYields)
forecastTermStructure = ql.RelinkableYieldTermStructureHandle()
forecastTermStructure.linkTo(oisZeroYields)

In [119]:

# Assuming mc is an object with calendar and fixedSwapConvention attributes
start_date = mc.calendar.advance(today, ql.Period(2, ql.Days), mc.fixedSwapConvention)

maturity_date = mc.calendar.advance(start_date, ql.Period(10, ql.Years), mc.fixedSwapConvention)

freq = ql.Monthly



In [ ]:
def testLoan(start_date, maturity_date, freq):
    as_of_date = ql.Settings.instance().evaluationDate

    eonia_index = ql.Eonia(ql.YieldTermStructureHandle(oisZeroYields))
    libor3m_index = ql.Euribor3M(ql.YieldTermStructureHandle(oisZeroYields))


    spreaded_discount_curve = ql.ZeroSpreadedTermStructure(
    ql.YieldTermStructureHandle(oisZeroYields),
    ql.QuoteHandle(ql.SimpleQuote(0.01) )) # 1% credit spread

    bond_engine = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(spreaded_discount_curve))

    optionlet_vol = ql.ConstantOptionletVolatility(
        0,
        ql.TARGET(),
        ql.ModifiedFollowing,
        0.1,
        ql.Actual360()
    )

    coupon_pricer = ql.BlackIborCouponPricer(ql.OptionletVolatilityStructureHandle(optionlet_vol))
    schedule = ql.Schedule(start_date, maturity_date, ql.Period(freq), mc.calendar,
                           ql.Unadjusted, ql.ModifiedFollowing, ql.DateGeneration.Forward, False)
    notional = 100000
    notional_schedule = [notional] * (len(schedule) - 1)



    gearings= [],
    spreads = [],
    caps = [0.05],
    floors = [0],

    floater = ql.AmortizingFloatingRateBond(mc.settlementDays, notional_schedule, schedule,
                                            libor3m_index, ql.Actual360(), ql.ModifiedFollowing,
                                            mc.settlementDays, [], [], [], [], True, as_of_date)
    
    


    




    floater.setPricingEngine(bond_engine)
    ql.setCouponPricer(floater.cashflows(), coupon_pricer)

    price = floater.NPV()
    print(f"NPV: {price}")

    #yr = floater.yield(ql.Actual360(), ql.Compounded, ql.Semiannual)
    #floater.duration(ql.Actual360(), yield_rate, ql.Compounded, ql.Semiannual, ql.Duration.Modified)
    #floater.convexity(ql.Actual360(), yield_rate, ql.Compounded, ql.Semiannual)

    
    # Extract cashflow data

    




    data = []
    for cf in floater.cashflows():
        row = {
            "Payment Date": cf.date().ISO(),
            "Amount": cf.amount(),
            "Accrual Start": None,
            "Accrual End": None,
            "Nominal": None,
            "Rate": None,
            "Spread": None,
            "Gearing": None
        }

        if ql.as_floating_rate_coupon(cf):
            frc = ql.as_floating_rate_coupon(cf)
            row.update({
                "Accrual Start": frc.accrualStartDate().ISO(),
                "Accrual End": frc.accrualEndDate().ISO(),
                "Nominal": frc.nominal(),
                "Rate": frc.rate(),
                "Spread": frc.spread(),
                "Gearing": frc.gearing()
            })
        elif ql.as_overnight_indexed_coupon(cf):
            oic = ql.as_overnight_indexed_coupon(cf)

            row.update({
                "Accrual Start": oic.accrualStartDate().ISO(),
                "Accrual End": oic.accrualEndDate().ISO(),
                "Nominal": oic.nominal(),
                "Rate": oic.rate(),
                "Spread": oic.spread(),
                "Gearing": oic.gearing()
            })
       

        data.append(row)


    # Create DataFrame
    df = pd.DataFrame(data)
    print(df)
    





In [129]:
testLoan(start_date,maturity_date,freq)

NPV: 90631.87068841388
    Payment Date         Amount Accrual Start Accrual End   Nominal      Rate  \
0     2025-09-15      -8.218865    2025-08-14  2025-09-14  100000.0 -0.000954   
1     2025-10-14      -8.856164    2025-09-14  2025-10-14  100000.0 -0.001063   
2     2025-11-14     -10.214518    2025-10-14  2025-11-14  100000.0 -0.001186   
3     2025-12-15     -10.584637    2025-11-14  2025-12-14  100000.0 -0.001270   
4     2026-01-14     -11.500363    2025-12-14  2026-01-14  100000.0 -0.001336   
..           ...            ...           ...         ...       ...       ...   
116   2035-05-14      62.986436    2035-04-14  2035-05-14  100000.0  0.007558   
117   2035-06-14      60.660250    2035-05-14  2035-06-14  100000.0  0.007044   
118   2035-07-16      54.283006    2035-06-14  2035-07-14  100000.0  0.006514   
119   2035-08-14      51.953483    2035-07-14  2035-08-14  100000.0  0.006033   
120   2035-08-14  100000.000000          None        None       NaN       NaN   

    